In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns;
import nltk 
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from time import time

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
from nltk.corpus import stopwords
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None
from nltk.tokenize import word_tokenize
from nltk import sent_tokenize
import unicodedata
import re
import inflect
from nltk.corpus import stopwords


In [0]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':'1gyCSLmU5zmpwagSePENbFWSoorTyNwdg'}) # replace the id with id of file you want to access
downloaded.GetContentFile('claim_extraction_18_10_2019_annotated.csv') 

In [0]:
df = pd.read_csv('claim_extraction_18_10_2019_annotated.csv')

In [0]:
#### Pour la classification True vs False
df_test1 = df[df['rating_alternateName']=="True"]
df_test2 = df[df['rating_alternateName']=="true"]
df_test = df[df['rating_alternateName']=="False"]
df_test3 = df[df['rating_alternateName']=="false"]
frames = [df_test1, df_test, df_test2, df_test3]
df_true_false = pd.concat(frames)
print(df_true_false.shape)


In [0]:
df_test1 = df[df['extra_entities_body']!="[]"]
frames = [df_test1]
df_extra_entities_body_delete_empty = pd.concat(frames)

In [0]:
print("Pour la classification True vs False\n")
total = df_true_false.isnull().sum().sort_values(ascending=False)
percent = (df_true_false.isnull().sum()/df_true_false.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

dispaly(missing_data)


#***Filtrage des colonnes***

On remarque que claimReview_author et claimReview_author_url sont totalement incomplètes il y a donc toutes les valeurs manquantes. On remarque aussi les colonnes qui sont très incomplètes: rating_bestRating,rating_ratingValue, rating_worstRating, creativeWork_author_sameAs, creativeWork_author_name et creativeWork_datePublished.

On remarque aussi que claimReview_author_name et claimReview_source sont identiques donc on garde une seul colonne

In [0]:
df2 = df_true_false.drop(columns=['claimReview_author','claimReview_author_name', 'claimReview_author_url', 'extra_tags', 'rating_bestRating', 'rating_ratingValue', 'rating_worstRating', 'creativeWork_author_sameAs', 'creativeWork_author_name', 'creativeWork_datePublished','claimReview_datePublished' ,'extra_entities_author', 'extra_entities_keywords','extra_refered_links', 'extra_entities_claimReview_claimReviewed','extra_entities_body' , 'extra_body'])
display(df2.shape)


In [0]:
df2 = df2[df2['claimReview_claimReviewed'].notnull()]


# ***pré-traitements rating_alternateName***

In [0]:
for column_rating_alternateName in df[['rating_alternateName']]:
   columnResult = df2[column_rating_alternateName]
   print('Column rating_alternateName Contents : ', columnResult.values)

   #Mise en minuscule
   tstringsObjects = [str(w) for w in columnResult.values] 
   tokens = [s.lower() for s in tstringsObjects] 
   print ("\naffichage de la colonne rating_alternateName aprés mise en minuscule : ", tokens)

tokens = np.array(tokens)
df2["rating_alternateName"] = tokens


#on donne le label 1 à true et 0 à false => ça va nous servir pour la classification
LabelValues = [1 if b=="true" else 0 for b in tokens]
LabelValues = np.array(LabelValues)
print("\naprés avoir donner des labels voici les résultats : ")
print ("nombre de valeur true : " , np.sum(LabelValues==1))
print ("nombre de valeur false : " , np.sum(LabelValues==0))

df2["rating_alternateName"] = LabelValues
#display(df2)

Column rating_alternateName Contents :  ['True' 'True' 'True' ... 'false' 'false' 'false']

affichage de la colonne rating_alternateName aprés mise en minuscule :  ['true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'tr

In [0]:
#Ponctuation
Punctuation_a_Supprime=re.sub(r'[-.]','',string.punctuation)
def Supprime_punctuation(text):
    text=text.translate(str.maketrans('', '', Punctuation_a_Supprime))
    return re.sub(r'(?<!\d)[.,;:](?![\d\sa-zA-Z])','',text)

#supprime Removal stopwords
def Supprime_stopwords(text):
    return " ".join([word for word in str(text).split() if word.lower() not in STOPWORDS])

#minuscule
def mise_en_miniscule(text):
        return " ".join([word.lower() for word in str(text).split() ])

#Lemmatization
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])



# ***pré-traitements claimReview_claimReviewed***

In [0]:
df2["claimReview_claimReviewed-pré-traitement"] = df2["claimReview_claimReviewed"].astype(str)  # chaine de carater str
df2["claimReview_claimReviewed-pré-traitement"] = df2["claimReview_claimReviewed-pré-traitement"].apply(lambda text: Supprime_punctuation(text))
df2["claimReview_claimReviewed-pré-traitement"] = df2["claimReview_claimReviewed-pré-traitement"].apply(lambda text: Supprime_stopwords(text))
df2["claimReview_claimReviewed-pré-traitement"] = df2["claimReview_claimReviewed-pré-traitement"].apply(lambda text: mise_en_miniscule(text))
df2["claimReview_claimReviewed-pré-traitement"] = df2["claimReview_claimReviewed-pré-traitement"].apply(lambda text: lemmatize_words(text))

# ***pré-traitements claimReview_source ***

In [0]:
df2["claimReview_source-pré-traitement"] = df2["claimReview_source"].astype(str)  # chaine de carater str
df2["claimReview_source-pré-traitement"]  = df2["claimReview_source-pré-traitement"].apply(lambda text: Supprime_punctuation(text))
df2["claimReview_source-pré-traitement"] = df2["claimReview_source-pré-traitement"].apply(lambda text: Supprime_stopwords(text))
df2["claimReview_source-pré-traitement"] = df2["claimReview_source-pré-traitement"].apply(lambda text: mise_en_miniscule(text))
df2["claimReview_source-pré-traitement"] = df2["claimReview_source-pré-traitement"].apply(lambda text: lemmatize_words(text))


# ***pré-traitements claimReview_claimReview_url***





In [0]:
from urllib.parse import urlparse
df2["claimReview_url-pré-traitement"] = df2["claimReview_url"].astype(str)  # chaine de carater str    
#df2["claimReview_url-pré-traitement"] = df2["claimReview_url-pré-traitement"].apply(lambda text: mise_en_miniscule(text))
#df2["claimReview_url-pré-traitement"] = df2["claimReview_url-pré-traitement"].apply(lambda text: lemmatize_words(text))
for column_claimReview_url in df2[['claimReview_url-pré-traitement']]:
   columnResult = df2[column_claimReview_url]

urls = []
minimized_urls = []
tstringsObjects = [str(w) for w in columnResult.values] 

print(len(tstringsObjects))
for i in range(len(tstringsObjects)):
  url = urlparse(tstringsObjects[i])
  urls.append(url)
for j in range(len(urls)):
  minimized_urls.append(urls[j][1])



15580


# ***pré-traitements extra_title***





In [0]:
df2["extra_title-pré-traitement"] = df2["extra_title"].astype(str)  # chaine de carater str
df2["extra_title-pré-traitement"] = df2["extra_title-pré-traitement"].apply(lambda text: Supprime_punctuation(text))
df2["extra_title-pré-traitement"] = df2["extra_title-pré-traitement"].apply(lambda text: Supprime_stopwords(text))
df2["extra_title-pré-traitement"] = df2["extra_title-pré-traitement"].apply(lambda text: mise_en_miniscule(text))
df2["extra_title-pré-traitement"] = df2["extra_title-pré-traitement"].apply(lambda text: lemmatize_words(text))

suppression des colonnes qui sont traite

In [0]:
df2=df2.drop('claimReview_claimReviewed',1)
df2=df2.drop('claimReview_source',1)
df2=df2.drop('claimReview_url',1)
#df2=df2.drop('extra_body',1)
df2=df2.drop('extra_title',1)

Retrouver l'ordre des colonne  et leur nom d'origine

In [0]:
df2["claimReview_claimReviewed"] = df2["claimReview_claimReviewed-pré-traitement"].astype(str) 
df2["claimReview_source"] = df2["claimReview_source-pré-traitement"].astype(str) 
df2["claimReview_url"] = minimized_urls 
#df2["extra_body"] = df2["extra_body-pré-traitement"].astype(str)
df2["extra_title"] = df2["extra_title-pré-traitement"].astype(str)  
df2["rating_alternateNamedeplace"] = df2["rating_alternateName"].astype(int)
display(df2["claimReview_url"].sample(5))

10868        www.snopes.com
19312    www.politifact.com
8886         www.snopes.com
5184         www.snopes.com
13540        www.snopes.com
Name: claimReview_url, dtype: object

In [0]:
df2=df2.drop('claimReview_claimReviewed-pré-traitement',1)
df2=df2.drop('claimReview_source-pré-traitement',1)
df2=df2.drop('claimReview_url-pré-traitement',1)
#df2=df2.drop('extra_body-pré-traitement',1)
df2=df2.drop('extra_title-pré-traitement',1)
#df2=df2.drop('extra_tags',1)
df2=df2.drop('rating_alternateName',1)

In [0]:
#df2["extra_tags"] = df2["extra_tagsdepl"]
df2["rating_alternateName"] = df2["rating_alternateNamedeplace"]

In [0]:
#df2=df2.drop('extra_tagsdepl',1)
df2=df2.drop('rating_alternateNamedeplace',1)

pour faire une bonne analyses de données on va supprime les lignes ou se trouve des information manquante  car extra_tags a des donnes manquante

In [0]:
Notredf= df2.dropna(axis=0)

In [0]:
Notredf

,Unnamed: 0,claimReview_claimReviewed,claimReview_source,claimReview_url,extra_title,rating_alternateName
4,4,september 2019 u.s. president donald trump sig...,snopes,www.snopes.com,donald trump sign 1.8 billion autism-services ...,1
27,27,u.s. sen. lindsey graham argued 1999 crime was...,snopes,www.snopes.com,u.s. sen. lindsey graham say crime isn’t requi...,1
43,43,fox news guest disparaged teenage climate acti...,snopes,www.snopes.com,fox news guest disparage teen climate activist...,1
55,55,ted livingston ceo kik interactive announced s...,snopes,www.snopes.com,kik messenger app shutting,1
56,56,u.s. governor worn blackface actual black gove...,snopes,www.snopes.com,america governor worn blackface black governor,1
...,...,...,...,...,...,...
39044,505,president buhari appointed judge called bar at...,factcheckafp,factcheck.afp.com,president buhari appoint supreme court judge n...,0
39051,512,rotimi amaechi arrested,factcheckafp,factcheck.afp.com,nigeria’s transport minister arrested,0
39081,542,scientific study prove man’s dna stay inside w...,factcheckafp,factcheck.afp.com,scientific study show man’s dna stay inside wo...,0
39100,561,president donald trump commented ipob biafra un,factcheckafp,factcheck.afp.com,u president donald trump comment biafran separ...,0


In [0]:
#sns.heatmap(Notredf.isnull(), cbar=False)

On perdre pas beacauoups de données

# ***DOWNSANPLING***

In [0]:
from sklearn.utils import resample
print(Notredf['rating_alternateName'])
tokens = np.array(Notredf['rating_alternateName'])
print(tokens)
# return number of false and true values
print ("nombre de valeur true : " , np.sum(tokens==1))
print ("nombre de valeur false : " , np.sum(tokens==0))

# Separate majority and minority classes

df_majority = Notredf[Notredf.rating_alternateName==0]
df_minority = Notredf[Notredf.rating_alternateName==1]

# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,     # sample with replacement
                                 n_samples=4183,    # to match minority class
                                 random_state=123) # reproducible results

# Combine minority class with upsampled majority class                                 
df_downsampled = pd.concat([df_minority, df_majority_downsampled])

# Display new class counts
print(df_downsampled.rating_alternateName.value_counts())



display(df_downsampled)
print(df_downsampled.shape)


4        1
27       1
43       1
55       1
56       1
        ..
39044    0
39051    0
39081    0
39100    0
39180    0
Name: rating_alternateName, Length: 15580, dtype: int64
[1 1 1 ... 0 0 0]
nombre de valeur true :  4183
nombre de valeur false :  11397
1    4183
0    4183
Name: rating_alternateName, dtype: int64


,Unnamed: 0,claimReview_claimReviewed,claimReview_source,claimReview_url,extra_title,rating_alternateName
4,4,september 2019 u.s. president donald trump sig...,snopes,www.snopes.com,donald trump sign 1.8 billion autism-services ...,1
27,27,u.s. sen. lindsey graham argued 1999 crime was...,snopes,www.snopes.com,u.s. sen. lindsey graham say crime isn’t requi...,1
43,43,fox news guest disparaged teenage climate acti...,snopes,www.snopes.com,fox news guest disparage teen climate activist...,1
55,55,ted livingston ceo kik interactive announced s...,snopes,www.snopes.com,kik messenger app shutting,1
56,56,u.s. governor worn blackface actual black gove...,snopes,www.snopes.com,america governor worn blackface black governor,1
...,...,...,...,...,...,...
31510,13035,nearly 70 percent federal spending go towards ...,politifact,www.politifact.com,rep. todd young say social security medicare m...,0
15404,15404,monica lewinsky offered take one team perform ...,snopes,www.snopes.com,monica lewinsky offer ‘take one team’ get trum...,0
30662,12187,say ted strickland supported giving 150 billio...,politifact,www.politifact.com,pac blame ted strickland iran deal didnt vote,0
4488,4488,image show genuine antifa manual created distr...,snopes,www.snopes.com,‘antifa manual’ real,0


(8366, 6)


# ***Shuffle***

In [0]:
from sklearn.utils import shuffle

def shuffle_dataset(df):
    """
    shuffles dataset entries and reset indexes

    returns a new dataframe containing the shuffled dataset with the reset indexes"""

    shuffled_df = shuffle(df)
    shuffled_df.reset_index(inplace = True, drop = True)

    print('Head of shuffled dataset :')
    display(shuffled_df.head())

    return shuffled_df


df_downsampled = shuffle_dataset(df_downsampled)
display(df_downsampled)

Head of shuffled dataset :


,Unnamed: 0,claimReview_claimReviewed,claimReview_source,claimReview_url,extra_title,rating_alternateName
0,16825,bride jailed pelting husband wedding cake,snopes,www.snopes.com,old brawl chain,1
1,1236,claim ohio gov. john kasich say climate change...,politifact,www.politifact.com,convention billboard tout john kasichs view cl...,1
2,1636,actor kurt russell said never seen man dedicat...,snopes,www.snopes.com,kurt russell say president trump ‘relentless’ ...,0
3,962,government shuts military personnel continue s...,politifact,www.politifact.com,bill young say military pay would delayed gove...,1
4,841,say sen. mitch mcconnell no. 1 recipient contr...,politifact,www.politifact.com,mitch mcconnell receive money lobbyist member ...,1


,Unnamed: 0,claimReview_claimReviewed,claimReview_source,claimReview_url,extra_title,rating_alternateName
0,16825,bride jailed pelting husband wedding cake,snopes,www.snopes.com,old brawl chain,1
1,1236,claim ohio gov. john kasich say climate change...,politifact,www.politifact.com,convention billboard tout john kasichs view cl...,1
2,1636,actor kurt russell said never seen man dedicat...,snopes,www.snopes.com,kurt russell say president trump ‘relentless’ ...,0
3,962,government shuts military personnel continue s...,politifact,www.politifact.com,bill young say military pay would delayed gove...,1
4,841,say sen. mitch mcconnell no. 1 recipient contr...,politifact,www.politifact.com,mitch mcconnell receive money lobbyist member ...,1
...,...,...,...,...,...,...
8361,11542,german british front-line soldier sang carol e...,snopes,www.snopes.com,world war christmas truce,1
8362,1962,voter agree november approve one-cent sale tax...,politifact,www.politifact.com,rail opponent claim penny tax commuter rail wo...,1
8363,8653,prince said voted republican political party k...,snopes,www.snopes.com,prince politics,0
8364,2316,dont think rudy fred john mccain support marri...,politifact,www.politifact.com,he right others hedged,1


#***Nettoyage***

In [0]:



def remove_non_ascii(words):
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word) 
    return new_words

def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', str(word))
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

def clean_text(text):
    tokens = word_tokenize(text)
    tokens=normalize(tokens)
    text="".join([" "+i for i in tokens]).strip()
    return text

In [0]:
############### Column claimReview_claimReviewed #####################################
for column_claimReview_claimReviewed in df_downsampled[['claimReview_claimReviewed']]:
   columnResult = df_downsampled[column_claimReview_claimReviewed]
   
tstringsObjects = [str(w) for w in columnResult.values] 
tokens=[clean_text(s) for s in tstringsObjects] 
df_downsampled["claimReview_claimReviewed"] = tokens
#print(df_downsampled["claimReview_claimReviewed"])

In [0]:
############### Column claimReview_source #####################################
for column_claimReview_source in df_downsampled[['claimReview_source']]:
   columnResult = df_downsampled[column_claimReview_source]
   

tstringsObjects = [str(w) for w in columnResult.values] 
tokens=[clean_text(s) for s in tstringsObjects] 
df_downsampled["claimReview_source"] = tokens
#print(df_downsampled["claimReview_source"])

In [0]:
############### Column claimReview_url #####################################
for column_claimReview_url in df_downsampled[['claimReview_url']]:
   columnResult = df_downsampled[column_claimReview_url]
   

tstringsObjects = [str(w) for w in columnResult.values] 
tokens=[clean_text(s) for s in tstringsObjects] 
df_downsampled["claimReview_url"] = tokens
#print(df_downsampled["claimReview_url"])

In [0]:
############### Column extra_title #####################################
for column_extra_title in df_downsampled[['extra_title']]:
   columnResult = df_downsampled[column_extra_title]
   

tstringsObjects = [str(w) for w in columnResult.values] 
tokens=[clean_text(s) for s in tstringsObjects] 
df_downsampled["extra_title"] = tokens
#print(df_downsampled["extra_title"])

#***Bag Of Words***

In [0]:
from nltk.stem.porter import PorterStemmer
corpus_full = []
def to_corpus(column):
  corpus  = []
  pstem = PorterStemmer()
  for i in range(df_downsampled[column].shape[0]):
    text = re.sub("[^a-zA-Z]", ' ', df_downsampled[column][i])
    corpus.append(text)
    corpus_full.append(text)
  print("Corpus created successfully")  
  return corpus  

In [0]:
def to_dictionary(corpus):
  #Create our dictionary 
  uniqueWordFrequents = {}
  for text in corpus:
      for word in text.split():
          if(word in uniqueWordFrequents.keys()):
              uniqueWordFrequents[word] += 1
          else:
              uniqueWordFrequents[word] = 1         
  #Convert dictionary to dataFrame
  uniqueWordFrequents = pd.DataFrame.from_dict(uniqueWordFrequents,orient='index',columns=['Word Frequent'])
  uniqueWordFrequents.sort_values(by=['Word Frequent'], inplace=True, ascending=False)
  uniqueWordFrequents.head(10)
  return uniqueWordFrequents

############### Column claimReview_claimReviewed #####################################



In [0]:
corpus_claimReview_claimReviewed = to_corpus('claimReview_claimReviewed')

Corpus created successfully


In [0]:
uniqueWordFrequents_claimReview_claimReviewed = to_dictionary(corpus_claimReview_claimReviewed)
uniqueWordFrequents_claimReview_claimReviewed.head(10)

,Word Frequent
thousand,1242
one,1206
and,1109
trump,1004
president,993
global,933
two,930
say,897
donald,707
hundred,665


In [0]:
uniqueWordFrequents_claimReview_claimReviewed['Word Frequent'].unique()

array([1242, 1206, 1109, 1004,  993,  933,  930,  897,  707,  665,  651,
        635,  542,  533,  502,  494,  485,  472,  470,  469,  468,  464,
        460,  458,  455,  449,  448,  446,  428,  426,  425,  387,  332,
        326,  317,  303,  290,  269,  248,  239,  214,  208,  203,  202,
        201,  199,  197,  192,  187,  186,  182,  178,  176,  173,  169,
        164,  157,  155,  149,  147,  143,  138,  135,  133,  132,  129,
        127,  126,  125,  124,  120,  119,  117,  116,  115,  114,  113,
        112,  110,  109,  107,  106,  104,  103,  102,  101,  100,   99,
         97,   95,   94,   93,   92,   91,   90,   89,   88,   87,   86,
         85,   84,   83,   82,   81,   80,   79,   78,   77,   76,   75,
         74,   73,   72,   71,   70,   69,   68,   67,   65,   64,   63,
         62,   61,   60,   59,   58,   57,   56,   55,   54,   53,   52,
         51,   50,   49,   48,   47,   46,   45,   44,   43,   42,   41,
         40,   39,   38,   37,   36,   35,   34,   

In [0]:
uniqueWordFrequents_claimReview_claimReviewed = uniqueWordFrequents_claimReview_claimReviewed[uniqueWordFrequents_claimReview_claimReviewed['Word Frequent'] >= 5]
print(uniqueWordFrequents_claimReview_claimReviewed.shape)
uniqueWordFrequents_claimReview_claimReviewed

(2973, 1)


,Word Frequent
thousand,1242
one,1206
and,1109
trump,1004
president,993
...,...
valid,5
gorilla,5
ride,5
truman,5


############### Column claimReview_url #####################################

In [0]:
corpus_claimReview_url = to_corpus('claimReview_url')
#display(corpus_claimReview_url)

Corpus created successfully


In [0]:
uniqueWordFrequents_claimReview_url = to_dictionary(corpus_claimReview_url)
uniqueWordFrequents_claimReview_url = uniqueWordFrequents_claimReview_url[uniqueWordFrequents_claimReview_url['Word Frequent'] >= 5]
print(uniqueWordFrequents_claimReview_url.shape)
#uniqueWordFrequents_claimReview_url

(6, 1)


############### Column claimReview_source #####################################

In [0]:
corpus_claimReview_source = to_corpus('claimReview_source')
#display(corpus_claimReview_source)

Corpus created successfully


In [0]:
uniqueWordFrequents_claimReview_source = to_dictionary(corpus_claimReview_source)
uniqueWordFrequents_claimReview_source = uniqueWordFrequents_claimReview_source[uniqueWordFrequents_claimReview_source['Word Frequent'] >= 5]
print(uniqueWordFrequents_claimReview_source.shape)

(6, 1)


############### Column extra_title #####################################

In [0]:
corpus_extra_title = to_corpus('extra_title')
#display(corpus_extra_title)

Corpus created successfully


In [0]:
uniqueWordFrequents_extra_title = to_dictionary(corpus_extra_title)
uniqueWordFrequents_extra_title = uniqueWordFrequents_extra_title[uniqueWordFrequents_extra_title['Word Frequent'] >= 5]
print(uniqueWordFrequents_extra_title.shape)

(2177, 1)


#***VECTORISATION***

In [0]:
#df1 = pd.DataFrame(uniqueWordFrequents_claimReview_claimReviewed)
#df1['Word Frequent df1'] = df1['Word Frequent']
#df1 = df1.drop('Word Frequent',1)

#df2 = pd.DataFrame(uniqueWordFrequents_claimReview_url)
#df2['Word Frequent df2'] = df2['Word Frequent']
#df2 = df2.drop('Word Frequent',1)

#df3 = pd.DataFrame(uniqueWordFrequents_claimReview_source)
#df3['Word Frequent df3'] = df3['Word Frequent']
#df3 = df3.drop('Word Frequent',1)

#df4 = pd.DataFrame(uniqueWordFrequents_extra_title)
#df4['Word Frequent df4'] = df4['Word Frequent']
#df4 = df4.drop('Word Frequent',1)

#frames = [df1, df2, df3, df4]
#mydf = pd.concat(frames)
#display(mydf)

#counVec = CountVectorizer(max_features = uniqueWordFrequents_claimReview_claimReviewed.shape[0])
#df_vect1 = pd.DataFrame(counVec.fit_transform(corpus_claimReview_claimReviewed).toarray())
#counVec = CountVectorizer(max_features = uniqueWordFrequents_claimReview_url.shape[0])
#df_vect2 = pd.DataFrame(counVec.fit_transform(corpus_claimReview_url).toarray())
#counVec = CountVectorizer(max_features = uniqueWordFrequents_claimReview_source.shape[0])
#df_vect3 = pd.DataFrame(counVec.fit_transform(corpus_claimReview_source).toarray())
#counVec = CountVectorizer(max_features = uniqueWordFrequents_extra_title.shape[0])
#df_vect4 = pd.DataFrame(counVec.fit_transform(corpus_extra_title).toarray())

#frames = [df_vect1, df_vect2, df_vect3, df_vect4]
#df_features = pd.concat(frames)
#display(df_features)

In [0]:
#display(df1)
#df1 = df1.drop('Word Frequent df1',1)
#display(corpus_extra_title)
df7 = pd.DataFrame()
df7["corpus 1"] = corpus_claimReview_claimReviewed

df7['corpus 2']  = corpus_claimReview_source

df7['corpus 3'] = corpus_claimReview_url

df7['corpus 4'] = corpus_extra_title

mydf2 = df7
display(mydf2)

,corpus 1,corpus 2,corpus 3,corpus 4
0,bride jailed pelting husband wedding cake,snopes,wwwsnopescom,old brawl chain
1,claim ohio gov john kasich say climate change ...,politifact,wwwpolitifactcom,convention billboard tout john kasichs view cl...
2,actor kurt russell said never seen man dedicat...,snopes,wwwsnopescom,kurt russell say president trump relentless de...
3,government shuts military personnel continue s...,politifact,wwwpolitifactcom,bill young say military pay would delayed gove...
4,say sen mitch mcconnell one recipient contribu...,politifact,wwwpolitifactcom,mitch mcconnell receive money lobbyist member ...
...,...,...,...,...
8361,german british frontline soldier sang carol ex...,snopes,wwwsnopescom,world war christmas truce
8362,voter agree november approve onecent sale tax ...,politifact,wwwpolitifactcom,rail opponent claim penny tax commuter rail wo...
8363,prince said voted republican political party k...,snopes,wwwsnopescom,prince politics
8364,dont think rudy fred john mccain support marri...,politifact,wwwpolitifactcom,right others hedged


In [0]:
#vectors = vectorizer.fit_transform(mydf).toarray()
from sklearn_pandas import DataFrameMapper
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

counVec = CountVectorizer(max_features = uniqueWordFrequents_claimReview_claimReviewed.shape[0])
vectorizer1 = TfidfVectorizer(min_df=1, ngram_range=(1,2), max_features = uniqueWordFrequents_claimReview_claimReviewed.shape[0])

vectorizer2 = TfidfVectorizer(min_df=1, ngram_range=(1,2), max_features = uniqueWordFrequents_extra_title.shape[0])
#vectorizer = TfidfVectorizer(min_df=1, ngram_range=(1,2), max_features = df_downsampled.shape[0])

labelizer = LabelEncoder()
mapper = DataFrameMapper([
     ('corpus 1', vectorizer1),
     ('corpus 2', labelizer),
     ('corpus 3', labelizer),
     ('corpus 4', vectorizer2)
      ])

features = mapper.fit_transform(mydf2)
display(features)
print(features.shape)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

(8366, 5152)


In [0]:
#counVec = CountVectorizer(max_features = uniqueWordFrequents_claimReview_claimReviewed.shape[0])
#bagOfWords1 = counVec.fit_transform(corpus_claimReview_claimReviewed).toarray()
#display(bagOfWords1)
#display(bagOfWords1.shape)

In [0]:
#counVec = CountVectorizer(max_features = uniqueWordFrequents_claimReview_url.shape[0])
#bagOfWords2 = counVec.fit_transform(corpus_claimReview_url).toarray()
#display(bagOfWords2)
#display(bagOfWords2.shape)

In [0]:
#counVec = CountVectorizer(max_features = uniqueWordFrequents_claimReview_source.shape[0])
#bagOfWords3 = counVec.fit_transform(corpus_claimReview_source).toarray()
#display(bagOfWords3)
#display(bagOfWords3.shape)

In [0]:
#counVec = CountVectorizer(max_features = uniqueWordFrequents_extra_title.shape[0])
#bagOfWords4 = counVec.fit_transform(corpus_extra_title).toarray()
#display(bagOfWords4)
#display(bagOfWords4.shape)

#***Cross Validation***

In [0]:
from datetime import datetime
X = features
y = df_downsampled['rating_alternateName']
display(X.shape)
display(y.shape)

(8366, 5152)

(8366,)

In [0]:
models = {
    'LogisticRegression': LogisticRegression(dual=False, max_iter=120000),
    #'SGDClassifier': SGDClassifier(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'GaussianNB': GaussianNB(),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'LinearSVC': LinearSVC()
}

CV_SEED = 7 #seed used for random selection of partitions during cross validation
CV_SCORING = 'accuracy'

k_fold = KFold(n_splits=10, shuffle=True, random_state=CV_SEED)

for name, model in models.items():
    start_time = time()
    print('Cross validation started at {}'.format(datetime.now()))
    cv_score = cross_val_score(model, X, y, cv=k_fold, scoring=CV_SCORING)
    output = """
    Time taken to complete cross validation of {}: {} seconds
    Accuracy scores over 10 evaluations: {}
    Mean score: {}
    Standard deviation of scores: {}
    """.format(name, time() - start_time, cv_score, cv_score.mean(), cv_score.std())
    
    print(output)
    

Cross validation started at 2020-05-15 19:38:35.637429

    Time taken to complete cross validation of LogisticRegression: 88.4531626701355 seconds
    Accuracy scores over 10 evaluations: [0.70967742 0.70489845 0.68339307 0.69653524 0.70250896 0.70489845
 0.71052632 0.68062201 0.71291866 0.68899522]
    Mean score: 0.6994973789965301
    Standard deviation of scores: 0.010988685794778411
    
Cross validation started at 2020-05-15 19:40:04.091768

    Time taken to complete cross validation of DecisionTreeClassifier: 573.6617012023926 seconds
    Accuracy scores over 10 evaluations: [0.63918757 0.68219833 0.65830346 0.67144564 0.68458781 0.66427718
 0.65430622 0.67822967 0.67105263 0.65311005]
    Mean score: 0.6656698564593302
    Standard deviation of scores: 0.013767925500013083
    
Cross validation started at 2020-05-15 19:49:37.754281

    Time taken to complete cross validation of RandomForestClassifier: 424.7713828086853 seconds
    Accuracy scores over 10 evaluations: [0.7013

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn


    Time taken to complete cross validation of LinearSVC: 12.248079538345337 seconds
    Accuracy scores over 10 evaluations: [0.68578256 0.67622461 0.65232975 0.66666667 0.6953405  0.67980884
 0.70334928 0.67703349 0.69019139 0.68421053]
    Mean score: 0.6810937616115884
    Standard deviation of scores: 0.013748085448273667
    


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
#Time taken to complete cross validation of LogisticRegression: 26.314778566360474 seconds
#    Accuracy scores over 10 evaluations: [0.68817204 0.71206691 0.70250896 0.7037037  0.66427718 0.70967742
# 0.68660287 0.70095694 0.68899522 0.67703349]
#    Mean score: 0.6933994729410689
#    Standard deviation of scores: 0.014382594796671632
#    
#Cross validation started at 2020-05-15 10:32:38.022654

#    Time taken to complete cross validation of SGDClassifier: 33.913071155548096 seconds
 #   Accuracy scores over 10 evaluations: [0.69295102 0.7037037  0.70609319 0.70489845 0.66427718 0.67980884
# 0.67583732 0.68421053 0.66028708 0.6722488 ]
#    Mean score: 0.6844316109596246
#    Standard deviation of scores: 0.01601552392716991
    
#Cross validation started at 2020-05-15 10:33:11.936741
#
#    Time taken to complete cross validation of DecisionTreeClassifier: 77.14127516746521 seconds
#    Accuracy scores over 10 evaluations: [0.66666667 0.65710872 0.71326165 0.63560335 0.66905615 0.65830346
# 0.67105263 0.67942584 0.65191388 0.67822967]
#    Mean score: 0.6680622009569378
 #   Standard deviation of scores: 0.019536978107019095
    
#Cross validation started at 2020-05-15 10:34:29.078685

#    Time taken to complete cross validation of RandomForestClassifier: 141.54014611244202 seconds
#    Accuracy scores over 10 evaluations: [0.72998805 0.70848268 0.73357228 0.69653524 0.69892473 0.69892473
# 0.68899522 0.7284689  0.72488038 0.71650718]
#    Mean score: 0.7125279392681769
#    Standard deviation of scores: 0.015382670074339093
    
#Cross validation started at 2020-05-15 10:36:50.619310

 #   Time taken to complete cross validation of GaussianNB: 3.67417311668396 seconds
 #   Accuracy scores over 10 evaluations: [0.66905615 0.68458781 0.69175627 0.69414576 0.65471924 0.69653524
 #0.66028708 0.6937799  0.66028708 0.66626794]
 #   Mean score: 0.677142248746663
 #   Standard deviation of scores: 0.015702374668163162
    
#Cross validation started at 2020-05-15 10:36:54.294002

 #   Time taken to complete cross validation of KNeighborsClassifier: 307.95891857147217 seconds
  #  Accuracy scores over 10 evaluations: [0.63201912 0.64516129 0.66188769 0.609319   0.61887694 0.63560335
 #0.62200957 0.63755981 0.63397129 0.62799043]
 #   Mean score: 0.6324398483991015
 #   Standard deviation of scores: 0.013837879259218113
    
#Cross validation started at 2020-05-15 10:42:02.253731
#/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
#  "the number of iterations.", ConvergenceWarning)
#/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
#  "the number of iterations.", ConvergenceWarning)

#    Time taken to complete cross validation of LinearSVC: 7.553427457809448 seconds
#    Accuracy scores over 10 evaluations: [0.65949821 0.69772999 0.68936679 0.69175627 0.64994026 0.69414576
# 0.66866029 0.66866029 0.67583732 0.66507177]
#    Mean score: 0.6760666941057433
#    Standard deviation of scores: 0.015514386975313044#

Les meilleurs candidats sont RandomForestClassifier & LogisticRegression

#***GridSearch***

In [0]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV

class GridSearchResult:

    def __init__(self, name, score, estimator):
        self.name = name
        self.score = score
        self.estimator = estimator

    def __str__(self):
        return """
        Model: {}
        Best Accuracy Score: {}
        Best Estimator: {}
        """.format(self.name, self.score, self.estimator)


#GRIDSEARCH USING THE ACCURACY METRIC FOR PARAMETERS TUNING
GRDSR_SCORING = 'accuracy'

#dictionary containing the candidate models that will be used 
#for parameters tuning using a GridSearchCV
candidates = {
    'LogisticRegression': models['LogisticRegression'],
    'RandomForestClassifier': models['RandomForestClassifier']
}

#dictionary of the hyperparameters to be tuned for each model
grid_params = {
    'LogisticRegression': {
        'C': np.logspace(-4,4,20),
        'penalty': ['l1','l2']
    },
    'RandomForestClassifier': { 
        'n_estimators': [200, 500],
        'max_features': ['auto', 'sqrt', 'log2'],
        'max_depth' : [4,5,6,7,8],
        'criterion' :['gini', 'entropy']
    }
}

#generation of training/test sets
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.20, random_state=55, shuffle=True)

#GridSearchCV for every candidate classifier
grid_search_results = []
for name, model in candidates.items():
    #creation of the gridsearch
    grd_sr = GridSearchCV(
        estimator = model,
        param_grid = grid_params[name],
        scoring = GRDSR_SCORING,
        cv = 5,
        n_jobs = -1,
        iid = True,
        return_train_score = True
    )
    
    #execution of the gridsearch
    start_time = time()
    print('Grid search started at {}'.format(datetime.now()))
    grd_sr.fit(X_train, y_train)
    print('\nTime taken to complete Grid search of {}: {} seconds'.format(name, time() - start_time))
    grd_sr_result = GridSearchResult(name, grd_sr.best_score_, grd_sr.best_estimator_)
    print(grd_sr_result)
    grid_search_results.append(grd_sr_result)

#sorting the results by descending order on the score column of the GridSearchResult objects
grid_search_results = sorted(grid_search_results, key=lambda result: result.score, reverse=True)
print('The best model with the best hyperparameters:\n{}'.format(grid_search_results[0]))

Grid search started at 2020-05-15 17:50:57.008070


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning



Time taken to complete Grid search of LogisticRegression: 1841.2359516620636 seconds

        Model: LogisticRegression
        Best Accuracy Score: 0.694560669456067
        Best Estimator: LogisticRegression(C=0.615848211066026, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=120000, multi_class='auto', n_jobs=None,
                   penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
                   verbose=0, warm_start=False)
        
Grid search started at 2020-05-15 18:21:38.247034


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning



Time taken to complete Grid search of RandomForestClassifier: 1858.0394682884216 seconds

        Model: RandomForestClassifier
        Best Accuracy Score: 0.6259713090257023
        Best Estimator: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=8, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
        
The best model with the best hyperparameters:

        Model: LogisticRegression
        Best Accuracy Score: 0.694560669456067
        Best Estimator: LogisticRegression(C=0.615848211066026, class_weight=None, dual=False,
                   fit_

#***CREATION DES PIPELINES***#

In [0]:
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
class DenseTransformer(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

In [228]:
#PIPELINE CREATION FOR LOGISTICREGRESSION CLASSIFIER
#creating the pipeline instance
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
#from sklearn.preprocessing import Imputer

lr_pipeline = make_pipeline(imputer, LogisticRegression(C=0.615848211066026, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=120000, multi_class='auto', n_jobs=None,
                   penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
                   verbose=0, warm_start=False))

#choosing data and target columns from initial dataset
X = df2[['claimReview_claimReviewed','claimReview_url','claimReview_source','extra_title']].notnull()
y = df2['rating_alternateName']

#generating the training/test sets from the initial dataset
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.20, random_state=55, shuffle=True)


#learning the model using the pipeline
start_time = time()
print('LogisticRegression classifier pipeline execution started at {}'.format(datetime.now()))
lr_pipeline.fit(X_train, y_train)
print('\nTime taken to complete pipeline execution: {} seconds'.format(time() - start_time))

#predicting the targets of test data
start_time = time()
print('\nLogisticRegression classifier prediction started at {}'.format(datetime.now()))
prediction_result = lr_pipeline.predict(X_test)
print('\nTime taken to complete prediction: {} seconds'.format(time() - start_time))

#accuracy, confusion matrix and classification report of the classifier
accuracy = accuracy_score(prediction_result, y_test)
conf = confusion_matrix(y_test, prediction_result)
report = classification_report(y_test, prediction_result)
print('''
Accuracy: {}
Confusion Matrix
{}

Classification Report
{}
'''.format(accuracy, conf, report))

LogisticRegression classifier pipeline execution started at 2020-05-15 21:05:24.667060

Time taken to complete pipeline execution: 0.04332780838012695 seconds

LogisticRegression classifier prediction started at 2020-05-15 21:05:24.710546

Time taken to complete prediction: 0.0021238327026367188 seconds

Accuracy: 0.7458279845956355
Confusion Matrix
[[2324    0]
 [ 792    0]]

Classification Report
              precision    recall  f1-score   support

           0       0.75      1.00      0.85      2324
           1       0.00      0.00      0.00       792

    accuracy                           0.75      3116
   macro avg       0.37      0.50      0.43      3116
weighted avg       0.56      0.75      0.64      3116




/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [238]:
lr_pipeline.fit(X_train , y_train)
predictions = lr_pipeline.predict(X_test)

print(predictions)

[0 0 0 ... 0 0 0]
